In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,45241
2,Huelva,Confirmados PDIA 14 días,366
3,Huelva,Tasa PDIA 14 días,"71,32139446966892"
4,Huelva,Confirmados PDIA 7 días,176
5,Huelva,Tasa PDIA 7 dias,"34,29662684880254"
6,Huelva,Total Confirmados,45444
7,Huelva,Curados,43941
8,Huelva,Fallecidos,412


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  45241.0


/tmp/ipykernel_74343/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12508.0


/tmp/ipykernel_74343/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_74343/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_74343/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_74343/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 3596 personas en los últimos 7 días 

Un positivo PCR cada 1653 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,45241.0,176.0,366.0,513170.0,34.296627,71.321394,43.0
Huelva-Costa,26783.0,106.0,229.0,289548.0,36.608783,79.088787,28.0
Condado-Campiña,14042.0,64.0,116.0,156231.0,40.964981,74.249029,14.0
Ayamonte,2066.0,23.0,43.0,21104.0,108.984079,203.752843,9.0
Huelva (capital),12508.0,40.0,87.0,143837.0,27.809256,60.485132,8.0
Lepe,3003.0,13.0,26.0,27880.0,46.628407,93.256815,5.0
Palos de la Frontera,1022.0,9.0,21.0,11742.0,76.647931,178.845171,4.0
Aljaraque,1620.0,6.0,9.0,21474.0,27.940766,41.911148,3.0
Almonte,2276.0,12.0,22.0,24507.0,48.965602,89.770270,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Higuera de la Sierra,70.0,4.0,6.0,1291.0,309.837335,464.756003,1.0
Paterna del Campo,207.0,4.0,14.0,3457.0,115.707261,404.975412,1.0
Ayamonte,2066.0,23.0,43.0,21104.0,108.984079,203.752843,9.0
Santa Bárbara de Casa,46.0,1.0,2.0,1043.0,95.877277,191.754554,0.0
Palos de la Frontera,1022.0,9.0,21.0,11742.0,76.647931,178.845171,4.0
Cartaya,1958.0,9.0,33.0,20083.0,44.814022,164.318080,2.0
Bollullos Par del Condado,1275.0,14.0,16.0,14387.0,97.310072,111.211510,2.0
Moguer,1950.0,11.0,21.0,21867.0,50.304111,96.035121,2.0
Isla Cristina,3112.0,8.0,20.0,21393.0,37.395410,93.488524,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Rociana del Condado,665.0,1.0,4.0,7939.0,12.596045,50.384179,0.0,0.250000
Cartaya,1958.0,9.0,33.0,20083.0,44.814022,164.318080,2.0,0.272727
Paterna del Campo,207.0,4.0,14.0,3457.0,115.707261,404.975412,1.0,0.285714
Sierra de Huelva-Andévalo Central,3992.0,6.0,21.0,67391.0,8.903266,31.161431,1.0,0.285714
Bonares,457.0,1.0,3.0,6060.0,16.501650,49.504950,0.0,0.333333
Isla Cristina,3112.0,8.0,20.0,21393.0,37.395410,93.488524,1.0,0.400000
Palos de la Frontera,1022.0,9.0,21.0,11742.0,76.647931,178.845171,4.0,0.428571
Huelva (capital),12508.0,40.0,87.0,143837.0,27.809256,60.485132,8.0,0.459770
Huelva-Costa,26783.0,106.0,229.0,289548.0,36.608783,79.088787,28.0,0.462882
